In [1]:
!pip install flask
!pip install pyserial
!jupyter nbconvert --to script functions.ipynb
!pip install nbimporter

[NbConvertApp] Converting notebook functions.ipynb to script
[NbConvertApp] Writing 5341 bytes to functions.py


In [12]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/')
def index():
    return 'Welcome to my capstone server'

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [10/May/2024 12:01:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/May/2024 12:01:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2024 12:01:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/May/2024 12:01:06] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
from flask import Flask
import serial
import threading
import subprocess
import nbimporter
from functions import detect_objects, crop_book, text_detection, text_correction

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM3', 9600)

# 인식된 텍스트를 저장할 전역 변수
recognized_texts_global = []
current_text_index = 0

def read_from_port(ser):
    while True:
        reading = ser.readline().decode().strip()  # 끝의 줄바꿈 제거
        print("Received from Arduino:", reading)
        if reading == "Take Picture":
            take_picture()
        elif reading == "Next":
            next_text()
        elif reading == "Previous":
            previous_text()
        else:
            print("Received from Arduino:", reading)

def take_picture():
    global recognized_texts_global  # 전역 변수 사용을 위한 선언
    global current_text_index  # 현재 텍스트 인덱스 초기화
    print("Taking picture...")
    img_path = "C:\\Users\\peter\\capstone\\camera\\test8.jpg"
    crop_book(img_path)
    img_left = "C:\\Users\\peter\\capstone\\onePage\\left_half.jpg"
    img_right = "C:\\Users\\peter\\capstone\\onePage\\right_half.jpg"
    recognized_texts = text_detection(img_left, img_right)
    corrected_texts = text_correction(recognized_texts)
    print(corrected_texts)
    # 공백이 아니고, 글자가 있는 요소만 필터링
    filtered_texts = [text for text in corrected_texts if text.strip()]
    print(filtered_texts)
    # 필터링된 텍스트를 전역 변수에 저장
    recognized_texts_global = filtered_texts  # 새로운 텍스트로 업데이트
    current_text_index = 0  # 새로운 텍스트 리스트로 업데이트되면 인덱스 초기화
    
def next_text():
    global current_text_index
    if current_text_index < len(recognized_texts_global) - 1:
        current_text_index += 1
        send_text_to_arduino(recognized_texts_global[current_text_index])
    else:
        print("End of the list")

def previous_text():
    global current_text_index
    if current_text_index > 0:
        current_text_index -= 1
        send_text_to_arduino(recognized_texts_global[current_text_index])
    else:
        print("Beginning of the list")

def send_text_to_arduino(text):
    print("Sending to Arduino:", text)
    ser.write((text + "\n").encode())
    
@app.route('/')
def index():
    return "Serial Communication with Arduino"

thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit


Received from Arduino: Take Picture
Taking picture...


Using cache found in C:\Users\peter/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-1 Python-3.12.2 torch-2.2.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Using CPU. Note: This module is much faster with a GPU.


["' ' A", '', '', '', "'", '', 'ONCE IN', 'A HOUSE ON EGYPT STREET', 'there lived', 'a', 'rabbit who', 'was', 'made almost entirely of', 'china', 'He had china', 'arms and china', 'china paws', 'and a china heads', 'a', 'china torso and', 'a', 'china nose His', 'arms and', 'were', 'jointed and joined by wire so that', 'his china elbows and china knees could be bent giving', 'him much freedom of movement', 'His ears were made of real rabbit fur and beneath the', 'fur there were strong bendable wires which allowed the', 'ears to be', 'arranged', "poses that reflected the rabbit's", 'mood jaunty tired full of ennui His tail too was', 'made of real rabbit fur and was fluffy and soft and well', 'china', '11', 'Chapter One', 'legs', 'legs', 'into']
["' ' A", "'", 'ONCE IN', 'A HOUSE ON EGYPT STREET', 'there lived', 'a', 'rabbit who', 'was', 'made almost entirely of', 'china', 'He had china', 'arms and china', 'china paws', 'and a china heads', 'a', 'china torso and', 'a', 'china nose His', '

127.0.0.1 - - [11/May/2024 20:46:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2024 20:46:12] "GET /favicon.ico HTTP/1.1" 404 -
